# Introduction

- Name : **Enggar Kristian**
- Recruitment Process : assignments that we've provide in link https://drive.google.com/drive/folders/1x5LvX3BAthgT2K8zpRq1faZTY84Fu9bC?usp=sharing.
- Company : PT. Sinar Mas Multiartha Tbk
- Role : Data Scientist/Data Engineer

## Objective : SOAL TEST PROGRAMMING

Using data above, use any programming language to clean the messy data with rules below:

- fullname can’t have symbol character except dot(.), But the dot(.) can’t be in the end of the sentence.
- DOB (Date of birth) must have format dd-mm-yyyy (day-month-year)
phone only consist of number and the beginning of the phone must be 0 (Format 62/+62 changed to 0).
- KTP only consist number.
- All of the data can’t have leading whitespace, trailing whitespace, tab (\t) and newline (\n)




# Import Libraries

**Note: for programming language python I used in Google Colab, so the data connect to my personal gdrive.**

In [1]:
# Come into my google colab
from google.colab import drive

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
!ls

gdrive	sample_data


In [3]:
# masuk folder "model_ml_dl"
%cd /content/gdrive/MyDrive/data_science_test/PT. Sinar Mas Multiartha Tbk/Task

/content/gdrive/MyDrive/data_science_test/PT. Sinar Mas Multiartha Tbk/Task


In [4]:
# Check folder 
!ls

data_cif_bad.csv
disaster_response_messages_test.csv
disaster_response_messages_training.csv
disaster_response_messages_validation.csv
suicide.csv


In [5]:
# Pandas libraries for dataframe
import pandas as pd

# Numpy for numeric
import numpy as np
pd.set_option ("display.max_columns", 10) 

# for preprocessing text
import re
import string

# format datetime
import datetime as dt

# Cleaning for phone number
!pip install phonenumbers
import phonenumbers
from phonenumbers import carrier, geocoder,timezone


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.6 MB 7.2 MB/s 


# Data Loading

In [97]:
df = pd.read_csv('data_cif_bad.csv',
                   sep = '|',
                   engine = 'python')
df

,cif_id,fullname,dob,phone,ktp,email,address
0,11111,Susi Susanti.,07-15-1993,+628782666330,3766.9987683.30004,susi_susanti@gmail.com,Jl. Bulutangkis No.7 RT. 003 RW. 006 Kel. Jom...
1,11112,Dr. Rudi Hartono.,10-22-1990,62876-5099-8431,32324-2311111-2463,hartono.rudi@yahoo.co.id,Jl. Harimau No.9 RT. 001 RW 002\t Kel. Sukamaj...
2,11113,Rani_Natalia-,08-17-1985,851993455690,36557498729O00002,rani.natalia@gmail.com,Jl. Mawar No.2 RT. 017 RW 005 \r\nKel. Cinere ...


In [98]:
# copy data
data_copy = df.copy()

In [99]:
# Want to know type each columns
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cif_id    3 non-null      int64 
 1   fullname  3 non-null      object
 2   dob       3 non-null      object
 3   phone     3 non-null      object
 4   ktp       3 non-null      object
 5   email     3 non-null      object
 6   address   3 non-null      object
dtypes: int64(1), object(6)
memory usage: 296.0+ bytes


# Preprocessing

In [100]:
# change column dob from object to date and will be format dd-mm-yyyy
data_copy['dob'] = pd.to_datetime(data_copy['dob']).dt.strftime('%d-%m-%Y')

In [101]:
data_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   cif_id    3 non-null      int64 
 1   fullname  3 non-null      object
 2   dob       3 non-null      object
 3   phone     3 non-null      object
 4   ktp       3 non-null      object
 5   email     3 non-null      object
 6   address   3 non-null      object
dtypes: int64(1), object(6)
memory usage: 296.0+ bytes


Dtype of column dob has change to datetime

In [102]:
# Cleaning the name
def clean_name(text):
  text = re.sub(r'[^\w]', ' ',text) # remove all symbols
  text = re.sub("[^a-zA-Z]", " ", text) # remove non-letters
  text = text.strip() # remove white space
  # adding dot in abbreviation text
  text = re.sub(r'(Dr)', r'\1.', text)
  text = re.sub(r'(Prof)', r'\1.', text)
  text = re.sub(r'(Mr)', r'\1.', text)
  text = re.sub(r'(Mrs)', r'\1.', text)
  text = re.sub(r'(Ms)', r'\1.', text)
  text = re.sub(r'(Jl)', r'\1.', text)
  text = re.sub(r'(No)', r'\1.', text)
  text = re.sub(r'(Kel)', r'\1.', text)
  text = re.sub(r'(Kec)', r'\1.', text)
  text = re.sub(" +" , " ", text)# remove double space into single space
  return text

# Cleaning the address
def clean_address(text):
  # text = re.sub(r'[^\w]', ' ',text) # remove all symbols
  text = re.sub(r"\n"," ",text) # remove \n
  text = re.sub(r"\t"," ",text) # remove \t
  text = re.sub(r"\r"," ",text) # remove \r
  # adding dot in abbreviation text
  # text = re.sub(r'(Jl)', r'\1.', text)
  # text = re.sub(r'(No)', r'\1.', text)
  # text = re.sub(r'(Kel)', r'\1.', text)
  # text = re.sub(r'(Kec)', r'\1.', text)
  # text = re.sub("No. ", "No.", text)# remove space after No.
  text = re.sub(" +" , " ", text)# remove double space into single space        
  return text

# Cleaning the phone numbers
def clean_phone(text):
  N = 1
  text = text.replace("-","")
  if text.startswith("62")==True:
    text = text.rjust(N + len(text), '+')
    print(str(text))
  elif text.startswith("8")==True:
    text = text.rjust(N + len(text), '2')
    text = text.rjust(N + len(text), '6')
    text = text.rjust(N + len(text), '+')
    print(str(text))
  else:
    print(text)
  text = phonenumbers.parse(text)
  text = phonenumbers.format_number(text, phonenumbers.PhoneNumberFormat.NATIONAL)
  text = text.replace("-", "")
  return text

# Cleaning text ktp
def clean_ktp(text):
  text = re.sub(r'[^\w]', ' ',text) # remove all symbols  text = text.replace
  text = text.replace(" ", "") # remove white space
  text = text.replace("O", "") # remove white space
  return text

In [103]:
# want to know origin of phone numbers
my_number = phonenumbers.parse("+6287786112081")
print("Country of origin\t: ", geocoder.description_for_number(my_number, "id"))
print("Provider in country\t: ",carrier.name_for_number(my_number, "id"))
print("Timezone\t\t: ", timezone.time_zones_for_number(my_number))

Country of origin	:  Indonesia
Provider in country	:  XL
Timezone		:  ('Asia/Jakarta',)


In [104]:
%%time
# The cleaning function applied in all reviews
data_copy['fullname'] = data_copy['fullname'].apply(lambda x: clean_name(x))
data_copy['phone'] = data_copy['phone'].apply(lambda x: clean_phone(x))
data_copy['ktp'] = data_copy['ktp'].apply(lambda x: clean_ktp(x))
data_copy['address'] = data_copy['address'].apply(lambda x: clean_address(x))


+628782666330
+6287650998431
+62851993455690
CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 9.59 ms


In [106]:
data_copy

,cif_id,fullname,dob,phone,ktp,email,address
0,11111,Susi Susanti,15-07-1993,08782666330,3766998768330004,susi_susanti@gmail.com,Jl. Bulutangkis No.7 RT. 003 RW. 006 Kel. Jomb...
1,11112,Dr. Rudi Hartono,22-10-1990,087650998431,3232423111112463,hartono.rudi@yahoo.co.id,Jl. Harimau No.9 RT. 001 RW 002 Kel. Sukamaju ...
2,11113,Rani Natalia,17-08-1985,0851993455690,3655749872900002,rani.natalia@gmail.com,Jl. Mawar No.2 RT. 017 RW 005 Kel. Cinere Kec....


Well all data has been cleaned.